In [349]:
import pandas as pd
import sqlite3

## • create a connection to the database using the library sqlite3

In [350]:
con = sqlite3.connect('../data/checking-logs.sqlite.sqlite')


## • using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
◦ time should have the values: after and before
◦ avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward
◦ only take into account the users that have observations before and after
## • we still are not using the lab ’project1’

In [351]:
# query = """ SELECT AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
#             FROM test JOIN deadlines ON test.labname = deadlines.labs
#                       JOIN (SELECT uid, COUNT(datetime) AS amount
#                             FROM pageviews
#                             WHERE datetime IN (SELECT DISTINCT datetime FROM pageviews)
#                             GROUP BY uid) AS counter ON test.uid = counter.uid
#             WHERE labname IS NOT 'project1' AND amount > 1
#         """

# query = """
#         SELECT AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
#         FROM test JOIN deadlines ON test.labname = deadlines.labs
#         WHERE labname IS NOT 'project1' AND uid IN (SELECT uid
#                     FROM pageviews
#                     GROUP BY uid
#                     HAVING COUNT(DISTINCT datetime) > 1
#                         )
#          AND first_commit_ts > first_view_ts
#         """

# query = """
#         SELECT
#         CASE
#         WHEN test.first_commit_ts < test.first_view_ts THEN 'before' ELSE 'after' END
#         AS time,
#         AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
#         FROM control JOIN deadlines ON control.labname = deadlines.labs
#         WHERE labname IS NOT 'project1'
#          AND first_commit_ts > first_view_ts
#         """

# query = """
#         SELECT AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
#         FROM control JOIN deadlines ON control.labname = deadlines.labs
#         WHERE labname IS NOT 'project1'
#         AND first_commit_ts < first_view_ts
#         AND (SELECT COUNT(DISTINCT datetime)
#                      FROM pageviews
#                      GROUP BY uid) > 1
#         """

# query = """
#         SELECT AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
#         FROM control JOIN deadlines ON control.labname = deadlines.labs
#         WHERE first_commit_ts > first_view_ts AND
#         uid IN (SELECT DISTINCT uid FROM test
#         WHERE uid IN (SELECT DISTINCT uid FROM test
#                       WHERE first_view_ts < first_commit_ts AND uid IS NOT 'project1')
#           AND uid IN (SELECT DISTINCT uid FROM test
#                       WHERE first_view_ts > first_commit_ts AND uid IS NOT 'project1')
#                 )
#         """

query = """
        SELECT AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
        FROM test JOIN deadlines ON test.labname = deadlines.labs
        WHERE first_commit_ts < first_view_ts AND
        uid IN (SELECT DISTINCT uid FROM test
        WHERE uid IN (SELECT DISTINCT uid FROM test
                       WHERE first_view_ts < first_commit_ts AND labname IS NOT 'project1')
        AND uid IN (SELECT DISTINCT uid FROM test
                       WHERE first_view_ts > first_commit_ts AND labname IS NOT 'project1')
         )
        """


# query = """
#         SELECT uid, datetime
#         FROM pageviews
#         GROUP BY uid
#
#         """
# query = """
#         SELECT DISTINCT uid FROM test
#         WHERE uid IN (SELECT DISTINCT uid FROM test
#                       WHERE first_view_ts < first_commit_ts AND uid IS NOT 'project1'
#                       )
#               AND uid IN (SELECT DISTINCT uid FROM test
#                       WHERE first_view_ts > first_commit_ts AND uid IS NOT 'project1'
#                       )
#         """
test_results = pd.io.sql.read_sql(query, con)
# test_results = pd.io.sql.read_sql("SELECT * FROM pageviews", con)
test_results

,avg_diff
0,-77.277778


## • close the connection

## • have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?